# 失控的提示词

本文来源于我对 [吴恩达的《面向开发者的提示工程》](https://datawhalechina.github.io/prompt-engineering-for-developers/#/C1/readme)
和 [OpenAI Prompt engineering](https://platform.openai.com/docs/guides/prompt-engineering) 的实践体会。


众所周知，提示词即是沟通的艺术，但沟通的对象不是人，而是大模型。提示词写的越好，大模型越会理解你的意图，输出符合你逾期的结果。
因此提示词在大模型应用开发中至关重要。

目前业界针对提示词总结了一套优化策略，比较知名的有吴恩达的《面向开发者的提示词工程》，最近我基于它和 ChatGPT 官方文档的提示词工程做了实践。
但发现效果不及预期，目前提示词工程对结果的把控能力较弱。远达不到“工程”的标准。

下面我会实测这些策略，并尝试进一步优化。


- 使用分隔符
- 结构化输出
- 条件判断
- Few-shot

- 让模型思考
  - 任务拆解、分步执行
  - 推理验证


In [114]:
from shallow_learning.tools.chatgpt import completion

## 使用分隔符


**用符号代替文字**


In [115]:
text = "在我的后园，可以看见墙外有两株树，一株是枣树，还有一株也是枣树。\
这上面的夜的天空，奇怪而高，我生平没有见过这样奇怪而高的天空，他仿佛要离开人间而去，使人们仰面不再看见。"

prompt_a = f"""
把用三个反引号括起来的文字总结成一句话。

```{text}```
"""

prompt_b = f"""
把用```括起来的文字总结成一句话。

```{text}```
"""

print(completion(prompt_a))
print("---")
print(completion(prompt_b))

在作者的后园，有两株枣树，夜晚的天空异常高远，仿佛要离开人间。
---
在后园，有两株枣树，夜空奇怪而高，仿佛要离开人间。


可以看到，把 “把用三个反引号括起来的文字总结成一句话。” 里的 “三个反引号” 替换为 “```” ，输出更加符合总结归纳的要求。


## 结构化输出


**用结构代替文字**

在这个例子里，提示词 `prompt_a` 和 `prompt_b` 会得到同样的输出，但使用结构化类型代替文字描述在遇到复杂类型的时候会对结果把握的更加精准。


In [116]:
# 吴恩达课件原 Prompt
prompt_a = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""

prompt_b = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，格式如下：

[{{
    book_id: int,
    title: str,
    author: str,
    genre: str
}}]
"""

print(completion(prompt_b))

[
  {
    "book_id": 1,
    "title": "时间之门",
    "author": "张三",
    "genre": "科幻"
  },
  {
    "book_id": 2,
    "title": "红楼梦续",
    "author": "李四",
    "genre": "文学"
  },
  {
    "book_id": 3,
    "title": "魔法学院",
    "author": "王五",
    "genre": "奇幻"
  }
]


**条件判断**


下面是我结合以上策略写的一个新例子：


In [117]:
food_a = "披萨"
food_b = "苹果，葡萄"

prompt = f"""
如果```中的文字是水果，那么输出它的主要产地和描述信息。
输出结果是 JSON 格式，格式如下：
[{{
    "origin": 主要产地,
    "descr": 描述信息
}}]
如果```中的文字不是水果，输出空值 None。

```{food_b}```
"""

print(completion(prompt))

[{
    "origin": "中国",
    "descr": "苹果是一种常见的水果，果实呈球形或椭圆形，果皮光滑，果肉多汁，味甜或酸甜。葡萄是一种著名的水果，果实呈圆形或椭圆形，果皮薄而有弹性，果肉多汁，味甜或酸甜。"
}]


一个细节：

在写条件判断的时候我们没有写 “如果...否则...” 而是写了 “如果...如果...” 你应该已经想到了原因，那就是如果 prompt 上下文很长，使用 `if else` 的形式，
大模型很可能匹配不到哪些部分是具体的条件逻辑，而 `if if` 这种格式则可以把条件逻辑分成比较小的独立的模块，更利于大模型理解。


**Few-shot**


目前大模型对示例的理解很可能和你的期望完全不一致。看下面这个例子，输出风格和原风格可以说是相距甚远。如果想提高输出准确度，提高示例数量或提供更详细的格式可以改善。


In [118]:
prompt_a = f"""
您的任务是以一致的风格回答问题。

<孩子>: 请教我何为耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 请教我何为韧性。
"""

prompt_b = f"""
请以一致的风格回答问题，注意比喻和排比的运用：

<孩子>: 请教我何为耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 请教我何为韧性。
"""

print(completion(prompt_a))
print("---")
print(completion(prompt_b))

<祖父母>: 韧性是指在面对困难和挑战时，能够坚持不懈、不放弃，持续努力并逐渐克服困难的能力。就像一棵树在风雨中弯曲但不折断，最终能够茁壮成长一样。
---
<祖父母>: 韧性如同一根弹簧，能够承受重压而不断回弹；如同一棵树，能够屹立不倒，经受住风雨的考验；如同一只强壮的马，能够跨越险阻，不停前行。


## 让模型思考


**拆解任务，分步执行**


In [119]:
text = "在我的后园，可以看见墙外有两株树，一株是枣树，还有一株也是枣树。\
这上面的夜的天空，奇怪而高，我生平没有见过这样奇怪而高的天空，他仿佛要离开人间而去，使人们仰面不再看见。"

prompt = f"""
步骤一：一句话总结```括起来的文字”。
步骤二：将步骤一的摘要翻译成日语”。

```{text}```
"""

print(completion(prompt))

步骤一：两株枣树，奇怪高的夜空。
步骤二：二つの枣の木、奇妙で高い夜空。


In [120]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
prompt = f"""
按以下格式输出：
摘要：一句话总结```括起来的文字，保留人名。
翻译：将摘要翻译成英语。
名称：英语摘要中的人名列表。
JSON：包含人名列表、人名数量和英文摘要的 JSON 对象。

JSON 对象结构如下：
{{
    names: List[str]
    num_names: int,
    English_summary: str
}}

```{text}```
"""
print(completion(prompt))

摘要：兄妹杰克和吉尔在山顶井里打水时发生意外，但仍然充满冒险精神。
翻译：Jack and Jill, siblings from a charming village, went to fetch water from a well on a mountaintop. While singing joyfully and climbing up, unfortunately, Jack stumbled on a rock and tumbled down the mountain, with Jill following closely behind. Despite some minor injuries, they made it back to their cozy home. Despite the mishap, their adventurous spirit remained undiminished, and they continued to explore with joy.
名称：Jack, Jill
JSON：{"names": ["Jack", "Jill"], "num_names": 2, "English_summary": "Jack and Jill, siblings from a charming village, went to fetch water from a well on a mountaintop. While singing joyfully and climbing up, unfortunately, Jack stumbled on a rock and tumbled down the mountain, with Jill following closely behind. Despite some minor injuries, they made it back to their cozy home. Despite the mishap, their adventurous spirit remained undiminished, and they continued to explore with joy."}


**推理验证**

推理验证的策略是没问题的，但是基于目前大模型的推理计算能力，即便是示例也不能稳定推理出正确结果。需谨慎使用。
